Import libraries

In [1]:
import os 
import pickle
import matchms
import numpy as np
from tqdm import tqdm 
from pprint import pprint
from random import sample

import rdkit.Chem as Chem
from rdkit.Chem import AllChem

from sklearn.datasets import make_regression
from sklearn.feature_selection import r_regression

from ms2deepscore import MS2DeepScore
from ms2deepscore.models import load_model

from matchms import Spectrum
from matchms import calculate_scores
from matchms.importing import load_from_msp
from matchms.similarity import CosineGreedy

Matplotlib created a temporary cache directory at /tmp/matplotlib-vtzxzuom because the default path (/afs/csail.mit.edu/u/k/klingmin/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Settings

In [2]:
k = 10000
data_path = "/data/rbg/users/klingmin/projects/MS_processing/data/final/MS_merged_w_mol_annotations.msp"
deepms_model_path = "/data/rbg/users/klingmin/projects/MS_processing/models/deepms/ms2deepscore_model.pt"
results_cache_folder = "./results_cache"
if not os.path.exists(results_cache_folder): os.makedirs(results_cache_folder)

Helper Functions

In [3]:
def get_all_spectra(path):

    spectra = [] 
    i = 0 

    for s in tqdm(load_from_msp(path)):
        
        mapping = {"+": "positive",
                   "-": "negative"}
        
        s.set("ionmode", mapping[s.metadata["adduct"][-1]])
        spectra.append(s)

        i += 1
    
    return spectra

In [4]:
def tanimoto_similarity(list_a, list_b):
    # Ensure both lists are of equal length
    assert len(list_a) == len(list_b), "Both lists must be of equal length"
    
    # Compute the AND of the two lists (common '1's)
    intersection = sum([1 for a, b in zip(list_a, list_b) if a == 1 and b == 1])
    
    # Compute the number of '1's in each list
    sum_a = sum(list_a)
    sum_b = sum(list_b)
    
    # Calculate Tanimoto similarity
    similarity = intersection / (sum_a + sum_b - intersection)
    
    return similarity

In [5]:
def pairwise_tanimoto_similarity(data, key):

    n_records = len(data)
    score = np.zeros((n_records, n_records))

    for i in range(n_records):

        for j in range(i, n_records):

            if i == j: 
                score[i,j] = 1.0 
            else:
                fp1, fp2 = data[i].metadata[key], data[j].metadata[key]
                fp1 = [int(c) for c in fp1]
                fp2 = [int(c) for c in fp2]
                ts = tanimoto_similarity(fp1, fp2)
                score[i,j] = ts
                score[j,i] = ts
    
    return score

In [6]:
def pickle_data(data, path):
    with open(path, "wb") as f: 
        pickle.dump(data, f)

In [7]:
def load_pickle(path):
    with open(path, "rb") as f: 
        return pickle.load(f)

Read the data

In [8]:
data_all = get_all_spectra(data_path)
data = sample(data_all, k)
print(f"There are {len(data)} records.")

250025it [05:46, 720.76it/s] 

There are 10000 records.


Get pairwise scores

In [9]:
greedy_cs = CosineGreedy()
model = load_model(deepms_model_path)
ms2deepscore = MS2DeepScore(model)


/data/rbg/users/klingmin/anaconda3/envs/chem/lib/python3.12/site-packages/ms2deepscore/models/load_model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_settings 

The model version (2.0.0) does not match the version of MS2Deepscore (2.2.0), consider downloading a new model or changing the MS2Deepscore version


In [ ]:
# Greedy cosine similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_cosine_similarity.pkl")):
    pairwise_cosine_similarity_score = calculate_scores(data, data, greedy_cs, is_symmetric = True)
    pickle_data(pairwise_cosine_similarity_score, os.path.join(results_cache_folder, "pairwise_cosine_similarity.pkl"))
else:
    pairwise_cosine_similarity_score = load_pickle(os.path.join(results_cache_folder, "pairwise_cosine_similarity.pkl"))

# Deepms similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_deepms_score.pkl")):
    pairwise_deepms_score = calculate_scores(data, data, ms2deepscore, is_symmetric = True)
    pickle_data(pairwise_deepms_score, os.path.join(results_cache_folder, "pairwise_deepms_score.pkl"))
else:
    pairwise_deepms_score = load_pickle(os.path.join(results_cache_folder, "pairwise_deepms_score.pkl"))

# maccs score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_maccs_score.pkl")):
    pairwise_maccs_score = pairwise_tanimoto_similarity(data, "maccs")
    pickle_data(pairwise_maccs_score, os.path.join(results_cache_folder, "pairwise_maccs_score.pkl"))
else:
    pairwise_maccs_score = load_pickle(os.path.join(results_cache_folder, "pairwise_maccs_score.pkl"))

# morgan4_256 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan4_256_score.pkl")):
    pairwise_morgan4_256_score = pairwise_tanimoto_similarity(data, "morgan4_256")
    pickle_data(pairwise_morgan4_256_score, os.path.join(results_cache_folder, "pairwise_morgan4_256_score.pkl"))
else:
    pairwise_morgan4_256_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan4_256_score.pkl"))

# morgan4_1024 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan4_1024_score.pkl")):
    pairwise_morgan4_1024_score = pairwise_tanimoto_similarity(data, "morgan4_1024")
    pickle_data(pairwise_morgan4_1024_score, os.path.join(results_cache_folder, "pairwise_morgan4_1024_score.pkl"))
else:
    pairwise_morgan4_1024_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan4_1024_score.pkl"))

# morgan4_2048 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan4_2048_score.pkl")):
    pairwise_morgan4_2048_score = pairwise_tanimoto_similarity(data, "morgan4_2048")
    pickle_data(pairwise_morgan4_2048_score, os.path.join(results_cache_folder, "pairwise_morgan4_2048_score.pkl"))
else:
    pairwise_morgan4_2048_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan4_2048_score.pkl"))

# morgan4_4096 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan4_4096_score.pkl")):
    pairwise_morgan4_4096_score = pairwise_tanimoto_similarity(data, "morgan4_4096")
    pickle_data(pairwise_morgan4_4096_score, os.path.join(results_cache_folder, "pairwise_morgan4_4096_score.pkl"))
else:
    pairwise_morgan4_4096_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan4_4096_score.pkl"))

# morgan6_256 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan6_256_score.pkl")):
    pairwise_morgan6_256_score = pairwise_tanimoto_similarity(data, "morgan6_256")
    pickle_data(pairwise_morgan6_256_score, os.path.join(results_cache_folder, "pairwise_morgan6_256_score.pkl"))
else:
    pairwise_morgan6_256_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan6_256_score.pkl"))

# morgan6_1024 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan6_1024_score.pkl")):
    pairwise_morgan6_1024_score = pairwise_tanimoto_similarity(data, "morgan6_1024")
    pickle_data(pairwise_morgan6_1024_score, os.path.join(results_cache_folder, "pairwise_morgan6_1024_score.pkl"))
else:
    pairwise_morgan6_1024_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan6_1024_score.pkl"))

# morgan6_2048 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan6_2048_score.pkl")):
    pairwise_morgan6_2048_score = pairwise_tanimoto_similarity(data, "morgan6_2048")
    pickle_data(pairwise_morgan6_2048_score, os.path.join(results_cache_folder, "pairwise_morgan6_2048_score.pkl"))
else:
    pairwise_morgan6_2048_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan6_2048_score.pkl"))

# morgan6_4096 score similarity
if not os.path.exists(os.path.join(results_cache_folder, "pairwise_morgan6_4096_score.pkl")):
    pairwise_morgan6_4096_score = pairwise_tanimoto_similarity(data, "morgan6_4096")
    pickle_data(pairwise_morgan6_4096_score, os.path.join(results_cache_folder, "pairwise_morgan6_4096_score.pkl"))
else:
    pairwise_morgan6_4096_score = load_pickle(os.path.join(results_cache_folder, "pairwise_morgan6_4096_score.pkl"))